# Brain translational and rotational normalization
* Zero z at the slice with max area
* Use analytic minimization of moment of inertia to find orientation angle

Generates:
* `train_dicom_diags_norm.csv` and `test_dicom_norm.csv` with 5 new columns added: 
  * `normz` normalized z
  * `xcm`, `ycm` center of mass
  * `theta` (rads) the CW angle the slice must be rotated to straighten it
  * `pct_tissue` percentage of brain tissue pixels in the radiography

In [1]:
stage = "stage_2"

In [2]:
import pydicom
import math
from pathlib import Path
from fastai.vision import *
from matplotlib import pyplot as plt
from scipy import ndimage, misc
from itertools import repeat
import pandas as pd
%matplotlib inline

In [3]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [4]:
def dcm_to_np(dcm):
    ''' Dicom to numpy array
    :param dcm: dicom object
    '''
    rescale_slope, rescale_intercept = float(dcm.RescaleSlope), float(dcm.RescaleIntercept)
    t = dcm.pixel_array.astype(np.float)
    t = t * rescale_slope + rescale_intercept # rescale
    return t

In [5]:
def get_fid_with_max_area(dir, dfs):
    ''' Given a single study return the fid (fileid) with biggest brain area
    :param dir: datset directory
    :param dfs: pandas dataframe with 1 study
    '''
    brain_area_vs_z = []
    pcts_tissue = []
    max_area = 0
    for index, row in dfs.iterrows():
        fid = row['SOPInstanceUID'] # don't do this inside f'' array or jupyter bugs out
        try:
            dcm = pydicom.dcmread(f"{dir}/{fid}.dcm")
            t = dcm_to_np(dcm)
        except Exception as e:
            print(e,fid)
        pts = np.argwhere(np.logical_and(0 < t, t < 100)) # select only brain matter
        if len(pts) > max_area:
            max_area = len(pts)
            fid_max_area = row['SOPInstanceUID']
        brain_area_vs_z.append([float(row['ImagePositionPatient_2']), len(pts)])
        pcts_tissue.append(len(pts) / t.size)
    return fid_max_area, pcts_tissue

In [6]:
def get_axis(brain):
    ''' 
    Get orientation axis.
    :param brain: numpy array (brain)
    :return: center of masses (xcm, ycm), coefficients (m,  n) of the axis equation y=mx+n and angle (th)
        the brain is rotated counter-clockwise (rotate clockwise to normalize)
    '''
    pts_org = np.argwhere(np.logical_and(0 < brain, brain < 100))

    # Center of masses
    xcm = pts_org[:,1].mean()
    ycm = pts_org[:,0].mean()

    # shift brain to center
    xs = pts_org[:,1] - xcm
    ys = pts_org[:,0] - ycm
    
    # coefficients of quad eq am^2+bm+c=0
    a = (xs * ys).sum()
    b = (xs ** 2 - ys ** 2).sum()
    c = -a

    # solve for m1 (max I), m2 (min I)
    m1 = (-b + (b**2-4*a*c) ** (.5)) / (2 * a)
    m2 = (-b - (b**2-4*a*c) ** (.5)) / (2 * a)
    
    # y-ycm = m*x - m*xcm -> y = m*x - m*xcm + ycm
    n1 = ycm - m1 * xcm
    n2 = ycm - m2 * xcm
    
    th = math.atan(m2)
    
    return xcm, ycm, m2, n2, th

In [7]:
def norm_study(dir, group_name):
    ''' Normalize study given by pandas DataFrame g
    :param dir: dataset directory
    :param g: single study to normalize
    '''
    global dfg
    g = dfg.get_group(group_name)
    fid_max_area, pcts_tissue = get_fid_with_max_area(dir, g)
    f = f'{dir}/{fid_max_area}.dcm'
    try:
        dcm = pydicom.dcmread(str(f))
    except Exceptione as e:
        print(e,f)
    brain = dcm_to_np(dcm)
    xcm, ycm, m, n, th = get_axis(brain)
    eg = g.copy()
    z0 = g[g['SOPInstanceUID'] == fid_max_area].iloc[0]['ImagePositionPatient_2']
    eg['normz'] = eg['ImagePositionPatient_2'] - z0
    eg['xcm'] = xcm
    eg['ycm'] = ycm
    eg['theta'] = th
    eg['pct_tissue'] = pcts_tissue
    return eg

# Train dataset

In [8]:
df = pd.read_csv(f'data/{stage}_train_dicom_diags.csv')

In [9]:
dfg = df.groupby('SeriesInstanceUID')

In [10]:
group_names = list(dfg.groups.keys())
#group_names = group_names[:64] # test with small subset

In [11]:
with ProcessPoolExecutor(max_workers=32) as e:
     extended_df = pd.concat(
         tqdm(e.map(norm_study, repeat(f'data/unzip/{stage}_train_images'), group_names), total=len(group_names)))

In [12]:
extended_df[90:100].T

,556142,561465,580918,584578,617029,617400,645424,661661,665938,748282
Unnamed: 0,556142,561465,580918,584578,617029,617400,645424,661661,665938,748282
BitsAllocated,16,16,16,16,16,16,16,16,16,16
BitsStored,16,16,16,16,16,16,16,16,16,16
Columns,512,512,512,512,512,512,512,512,512,512
HighBit,15,15,15,15,15,15,15,15,15,15
ImageOrientationPatient_0,1,1,1,1,1,1,1,1,1,1
ImageOrientationPatient_1,0,0,0,0,0,0,0,0,0,0
ImageOrientationPatient_2,0,0,0,0,0,0,0,0,0,0
ImageOrientationPatient_3,0,0,0,0,0,0,0,0,0,0
ImageOrientationPatient_4,0.979925,0.979925,0.979925,0.979925,0.979925,0.979925,0.979925,0.979925,0.979925,0.979925


In [13]:
assert len(df) == len(extended_df)
len(df)

752802

In [14]:
extended_df.to_csv(f'data/{stage}_train_dicom_diags_norm.csv')

In [15]:
df = pd.read_csv(f'data/{stage}_test_dicom.csv')

In [16]:
dfg = df.groupby('SeriesInstanceUID')

In [17]:
group_names = list(dfg.groups.keys())
len(group_names)

3518

In [18]:
df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
BitsAllocated,16,16,16,16,16
BitsStored,12,16,16,16,12
Columns,512,512,512,512,512
HighBit,11,15,15,15,11
ImageOrientationPatient_0,1,1,1,1,1
ImageOrientationPatient_1,0,0,0,0,0
ImageOrientationPatient_2,0,0,0,0,0
ImageOrientationPatient_3,0,0,0,0,0
ImageOrientationPatient_4,0.981627,0.987688,0.927184,0.986286,1


In [19]:
with ProcessPoolExecutor(max_workers=32) as e:
     extended_df = pd.concat(tqdm(
         e.map(norm_study, repeat(f'data/unzip/{stage}_test_images'), group_names), total=len(group_names)))

In [20]:
extended_df.head().T

,13820,18901,20132,24764,36224
Unnamed: 0,13820,18901,20132,24764,36224
BitsAllocated,16,16,16,16,16
BitsStored,12,12,12,12,12
Columns,512,512,512,512,512
HighBit,11,11,11,11,11
ImageOrientationPatient_0,1,1,1,1,1
ImageOrientationPatient_1,0,0,0,0,0
ImageOrientationPatient_2,0,0,0,0,0
ImageOrientationPatient_3,0,0,0,0,0
ImageOrientationPatient_4,0.939693,0.939693,0.939693,0.939693,0.939693


In [21]:
assert len(df) == len(extended_df)
len(df)

121232

In [22]:
extended_df.to_csv(f'data/{stage}_test_dicom_norm.csv')